In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets

In [ ]:
class Visualization:
    def __init__(self):
        self.setup()
        with self.output:
            self.game = Game(self)

    def setup(self):
        self.deal_button = widgets.Button(description="deal")
        self.deal_button.on_click(self.deal)

        self.reset_button = widgets.Button(description="reset", disabled=True)
        self.reset_button.on_click(self.reset)

        self.deck_size_label = widgets.Label(value="Deck size: -")

        self.player_cards_description = widgets.Label(value="Player cards:")
        self.player_cards_label = widgets.Label(value="-")
        self.player_cards_box = widgets.HBox([self.player_cards_description, self.player_cards_label])

        self.dealer_cards_description = widgets.Label(value="Dealer cards:")
        self.dealer_cards_label = widgets.Label(value="-")
        self.dealer_cards_box = widgets.HBox([self.dealer_cards_description, self.dealer_cards_label])


        self.hit_button = widgets.Button(description="hit", disabled=True)
        self.stand_button = widgets.Button(description="stand", disabled=True)
        self.double_button = widgets.Button(description="double", disabled=True)
        self.split_button = widgets.Button(description="split", disabled=True)

        
        self.output = widgets.Output()

        self.menu_box = widgets.HBox([self.deal_button, self.reset_button])

        self.action_box = widgets.HBox([self.hit_button, self.stand_button, self.double_button, self.split_button])

        display(self.menu_box, self.action_box, self.player_cards_box, self.dealer_cards_box, self.output)

    def deal(self, b):
        self.reset_button.disabled = False
        self.deal_button.disabled = True
        with self.output:
            self.game.play_round()
  
    def reset(self, b):
        with self.output:
            self.game = Game(self)
            clear_output()
        self.player_cards_label.value = "-"
        self.dealer_cards_label.value = "-"
        self.deck_size_label.value = "Deck size: -"
        self.reset_button.disabled = True
        self.deal_button.disabled = False
        self.disable_action_buttons()
        
    
    def disable_action_buttons(self):
        self.hit_button.disabled = True
        self.stand_button.disabled = True
        self.double_button.disabled = True
        self.split_button.disabled = True